## Passing Authority: Multisig accounts and Rekeying
#### 03.5 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-02

- Create and use k-out-of-n multisignature accounts
- Rekey an account

## Setup
Starting with chapter 3.4, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [5]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [6]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future.transaction import PaymentTxn, MultisigTransaction
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [7]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [8]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

VL5UU2QXNKNEH7VISHZFU2ALXN5MOIDD3KXEYX2ADLCCIYN3MCOKRBATV4
VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU
CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA
YXUPADIBAU7TOTY5MIOJIN6HDXS42SQCTMAPEUHU4GQCAKZQXJXXCQOJU4


## Multisig account
Alice, Bob and Charlie create a multisig account, 
with the requirement that at least 2 out of 3 have to sign.

#### Step 1: Create the Multisig account
* Using the new command `Multisig`

In [9]:
from algosdk.future.transaction import Multisig

# create a multisig account
version   = 1     # multisig version, currently always 1
threshold = 2     # min 2 signatures required

account_list = [Alice['public'], Bob['public'], Charlie['public']]

msig = Multisig(version, threshold, account_list)
msig_address = msig.address()

print("Multisig Address: ", msig_address)
print('https://testnet.algoexplorer.io/address/'+msig_address)

Multisig Address:  R7ZBEZUKN63KMLZPE2O65VJ54MVKQNJG2VXBEUTPTXEQSZXX2FVP77R2BY
https://testnet.algoexplorer.io/address/R7ZBEZUKN63KMLZPE2O65VJ54MVKQNJG2VXBEUTPTXEQSZXX2FVP77R2BY


#### Step 2: Fund the multisig account
* Can be funded via PaymentTxn
* Here we fund from a dispenser

In [12]:
print('https://dispenser.testnet.aws.algodev.network/?account='+msig_address)

https://dispenser.testnet.aws.algodev.network/?account=R7ZBEZUKN63KMLZPE2O65VJ54MVKQNJG2VXBEUTPTXEQSZXX2FVP77R2BY


#### Step 3.1: Create a simple payment transaction
* Together, they want to send 1.2 ALGOs to Dina

In [14]:
sp = algod_client.suggested_params()
amt = int(1.2*1E6)

txn = PaymentTxn(sender=msig_address,         # <-- From Multisig address
                 sp=sp, 
                 receiver=Dina['public'], 
                 amt = amt
                )

#### Step 3.2: Convert to a multisig object and sign it
* Alice and Bob sign the transaction

In [15]:
# Create multisig object
mtx = MultisigTransaction(txn, msig)

# Collect at least 2 signatures
mtx.sign(Alice['private'])
mtx.sign(Bob['private'])

#### Step 3.3: Send the transaction

In [16]:
txid = algod_client.send_transaction(mtx)
print("Send transaction with txID: {}".format(txid))

Send transaction with txID: OUGUQW2MD5RIOETJMJRIWKD6JT3HBUAGWCPVDTNWRESRJZXTXUOQ


#### Step 3.4: Wait for confirmation

In [17]:
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26067264.
Waiting for round 26067264 to finish.
Waiting for round 26067265 to finish.
Transaction OUGUQW2MD5RIOETJMJRIWKD6JT3HBUAGWCPVDTNWRESRJZXTXUOQ confirmed in round 26067266.


#### Check Dina's holdings

In [18]:
print('https://testnet.algoexplorer.io/address/'+Dina['public'])

https://testnet.algoexplorer.io/address/YXUPADIBAU7TOTY5MIOJIN6HDXS42SQCTMAPEUHU4GQCAKZQXJXXCQOJU4


#### Exercise
* Repeat steps 3.1-3.4 with Alice and Charlie signing the transaction
* Repeat steps 3.1-3.4 with Alice, Bob and Charlie signing 
* Repeat steps 3.1-3.4 with only Alice signing

## Rekeying
Case 1
* Imagine your private key has been compromised
* You have lots of assets and maybe ongoing smart contracts with this address
* So you want to "change the password"

Case 2
* Bob is very busy and wants Charlie to sign for him

#### Bob rekeys to Charlie
Rekeying is done via a simple payment transaction of 0 ALGOs, that has an additional rekey entry

In [19]:
# Step 1: prepare special transaction
sp = algod_client.suggested_params()
amt = 0                                          # <-- Send zero ALGOs

txn = PaymentTxn(sender = Bob['public'],         # <-- Bob wants to rekey
                 sp = sp, 
                 receiver = Bob['public'],       # <-- Bob wants to rekey
                 amt = amt,
                 rekey_to = Charlie['public']    # <-- Charlie should be able to sign for Bob
                )

In [20]:
# Steps 2+3: sign and send
stxn = txn.sign(Bob['private'])                 # <-- Bob signs one last time
txid = algod_client.send_transaction(stxn)

In [21]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26067297.
Waiting for round 26067297 to finish.
Waiting for round 26067298 to finish.
Transaction SAZA3VDUIVD7KSUZMXOQDA2A37NEBKEMU77SDOX5GS3HMJNZPJKQ confirmed in round 26067299.


#### Charlie signs for Bob
* Charlie makes a payment of 1.4 ALGO to Alice out of Bob's account

In [22]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(1.4*1E6)                                

txn = PaymentTxn(sender = Bob['public'],           # <-- From Bob
                 sp = sp, 
                 receiver = Alice['public'],       # <-- To Alice 
                 amt = amt
                 )

# Steps 2+3: sign and send
stxn = txn.sign(Charlie['private'])                # <-- Charlie now signs for Bob
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26067300.
Waiting for round 26067300 to finish.
Waiting for round 26067301 to finish.
Transaction IPVNCU7LDML2LA2LRTHWPRUXUBTACOWDTZF5IHJE25ZSJLQTBQ5Q confirmed in round 26067302.


#### Bob cannot sign for himself
* Bob wants to make a payment of 1.7 ALGO to Dina

In [23]:
# Step 1: prepare special transaction

sp = algod_client.suggested_params()
amt = int(1.7*1E6)                                

txn = PaymentTxn(sender = Bob['public'],         # <-- From Bob
                 sp = sp, 
                 receiver = Bob['public'],       # <-- To Dina 
                 amt = amt
                )

# Steps 2+3: sign and send
stxn = txn.sign(Bob['private'])                 # <-- Bob tries to sign
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

AlgodHTTPError: TransactionPool.Remember: transaction DPGMZ3RVCX5EY5X2F3IT2P6LLUXLISX63HMFJ3YBN6SNP3H5OLVQ: should have been authorized by BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA but was actually authorized by CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM

#### Charlie rekeys back to Bob
* Charlie rekeys the Bob account back to Bob
* Important step for the rest of the course

In [24]:
# Step 1: prepare special transaction
sp = algod_client.suggested_params()
amt = 0                                          # <-- Rekeying = send 0 ALGOs

txn = PaymentTxn(sender = Bob['public'],         # <-- The address which is rekeyed
                 sp = sp, 
                 receiver = Bob['public'],       # <-- The address which is rekeyed 
                 amt = amt,
                 rekey_to = Bob['public']        # <-- Bob should (again) be able to sign for himself
                )

In [25]:
# Steps 2+3: sign and send
stxn = txn.sign(Charlie['private'])              # <-- Charlie signs the rekeying
txid = algod_client.send_transaction(stxn)

In [26]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26067308.
Waiting for round 26067308 to finish.
Waiting for round 26067309 to finish.
Waiting for round 26067310 to finish.
Transaction G3UONIAEXD5PZMX7LK7EZAR64O7OHANZ57QLJKCP6LQXDIMS7IQA confirmed in round 26067311.


#### Exercise
* To test the rekeying, run the payment transactions above again
    * Charlie cannot sign for Bob anymore
    * Bob can sign again for himself

## Things that can and will go wrong
* Be sure if you trust the new signature holder
* Rekeying can be easily slipped into a payment transaction
* Wrong address or typo in rekey means you loose all your money
    * The protocol does not check whether you or someone has actually access to the rekey address